## KNN

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import gc
pd.options.display.max_columns = 70
pd.options.display.max_rows = 70

In [2]:
# If importing is giving you an issue, don't import mixed data type columns 
#good_import_cols1 = pd.Series(range(0,57))
#good_import_cols = good_import_cols1.drop([22,32,34,49,55])


In [39]:
# If importing is giving you an issue, only import good columns  
train_df1 = pd.read_csv("train_2016_v2.csv", parse_dates=["transactiondate"])
prop = pd.read_csv("properties_2016.csv") #, usecols=good_import_cols)

C:\Users\board\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
missing_df = prop.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]

missing_df['missing_ratio'] = missing_df['missing_count'] / prop.shape[0]
missing_df.loc[missing_df['missing_ratio']>0.75]

,column_name,missing_count,missing_ratio
2,architecturalstyletypeid,2979156,0.997970
3,basementsqft,2983589,0.999455
6,buildingclasstypeid,2972588,0.995769
9,decktypeid,2968121,0.994273
10,finishedfloor1squarefeet,2782500,0.932093
13,finishedsquarefeet13,2977545,0.997430
14,finishedsquarefeet15,2794419,0.936086
15,finishedsquarefeet50,2782500,0.932093
16,finishedsquarefeet6,2963216,0.992630
18,fireplacecnt,2672580,0.895272


Impute mean values where there are missing values  
Will impute for where less than 75% are missing 

In [41]:
train_df = train_df1.merge(prop, on='parcelid', how = 'left')
mean_values = train_df.mean(axis=0)
train_df_new = train_df.fillna(mean_values, inplace=True)
cols_to_drop = train_df.dtypes[train_df.dtypes != "float64"].index
train_df.dtypes[train_df.dtypes != "float64"].index
train_df2 = train_df.drop(['propertycountylandusecode', 'propertyzoningdesc', 'taxdelinquencyflag', 'hashottuborspa','fireplaceflag' ], axis = 1)
missing_df2 = train_df2.isnull().sum(axis=0).reset_index()
missing_df2.columns = ['count', 'val']
missing_df2[missing_df2['val'] > 0]


,count,val


In [42]:
print(train_df2.head())


   parcelid  logerror transactiondate  airconditioningtypeid  \
0  11016594    0.0276      2016-01-01               1.000000   
1  14366692   -0.1684      2016-01-01               1.816372   
2  12098116   -0.0040      2016-01-01               1.000000   
3  12643413    0.0218      2016-01-02               1.000000   
4  14432541   -0.0050      2016-01-02               1.816372   

   architecturalstyletypeid  basementsqft  bathroomcnt  bedroomcnt  \
0                  7.229885    713.581395          2.0         3.0   
1                  7.229885    713.581395          3.5         4.0   
2                  7.229885    713.581395          3.0         2.0   
3                  7.229885    713.581395          2.0         2.0   
4                  7.229885    713.581395          2.5         4.0   

   buildingclasstypeid  buildingqualitytypeid  calculatedbathnbr  decktypeid  \
0                  4.0               4.000000                2.0        66.0   
1                  4.0            

In [43]:
train_df2.columns

Index(['parcelid', 'logerror', 'transactiondate', 'airconditioningtypeid',
       'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr',
       'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'storytypeid',
       'threequarterbathnbr', 'typeconstructiontypeid', 'unitcnt',
       'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numbero

We know that some of the columns are categorical and should be treated as such  
We know that BuildingClassTypeID, TypeConstructionTypeID, ArchitecturalStyleTypeID, AirConditioningTypeID, HeatingOrSystemTypeID, PropertyLandUseTypeID, StoryTypeID



In [44]:
from sklearn.preprocessing import OneHotEncoder

In [45]:
one_hot_colnames = ["buildingclasstypeid", "typeconstructiontypeid", "architecturalstyletypeid", "airconditioningtypeid", 
                    "heatingorsystemtypeid", "propertylandusetypeid", "storytypeid"]

In [71]:
one_hot_df = pd.DataFrame()
for col_name in one_hot_colnames:
    temp_one_hot_var1 = OneHotEncoder().fit_transform(train_df2[col_name].values.reshape(-1,1))
    temp_one_hot_var = temp_one_hot_var1.toarray()
    temp_one_hot_var_df = pd.DataFrame(data=temp_one_hot_var)
    temp_one_hot_var_df.add_prefix((col_name + "_"))
    one_hot_df = pd.concat([one_hot_df, temp_one_hot_var_df] , axis = 1)
print("Data Frame Size: " ,one_hot_df.shape)

KeyError: 'buildingclasstypeid'

In [49]:
one_hot_df.head()

,0,0,1,2,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9,10,11,12,0,1,2,3,4,5,6,7,8,9,10,11,12,13,0
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


The dataset we will be using is train_df2. Let's take a last quick peak and check it for nulls 

In [50]:
train_df2 = train_df2.drop(one_hot_colnames, axis = 1)
train_df2 = pd.concat([train_df2, one_hot_df], axis =1)

In [70]:
train_df2.head()

,parcelid,logerror,transactiondate,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,...,5,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9,10,11,12,0,1,2,3,4,5,6,7,8,9,10,11,12,13,0
0,11016594,0.0276,2016-01-01,713.581395,2.0,3.0,4.000000,2.0,66.0,1347.974037,1684.0,1684.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,2.0,1.812005,345.521228,34280990.0,-118488536.0,7528.0,1.0,519.827657,1.0,1.0,1.0,6.037107e+07,12447.0,3101.0,31817.000000,96370.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,14366692,-0.1684,2016-01-01,713.581395,3.5,4.0,5.565407,3.5,66.0,1347.974037,2263.0,2263.0,1404.545455,2380.090067,1355.546674,2302.546318,6059.0,1.187884,3.0,2.000000,468.000000,33668120.0,-117677556.0,3643.0,1.0,519.827657,1.0,1.0,1.0,6.059052e+07,32380.0,1286.0,190646.540237,96962.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,12098116,-0.0040,2016-01-01,713.581395,3.0,2.0,4.000000,3.0,66.0,1347.974037,2217.0,2217.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,3.0,1.812005,345.521228,34136312.0,-118175032.0,11423.0,1.0,519.827657,1.0,1.0,1.0,6.037464e+07,47019.0,3101.0,275411.000000,96293.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,12643413,0.0218,2016-01-02,713.581395,2.0,2.0,4.000000,2.0,66.0,1347.974037,839.0,839.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,2.0,1.812005,345.521228,33755800.0,-118309000.0,70859.0,1.0,519.827657,1.0,1.0,1.0,6.037296e+07,12447.0,3101.0,54300.000000,96222.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,14432541,-0.0050,2016-01-02,713.581395,2.5,4.0,5.565407,2.5,66.0,1347.974037,2283.0,2283.0,1404.545455,2380.090067,1355.546674,2302.546318,6059.0,1.187884,2.0,2.000000,598.000000,33485643.0,-117700234.0,6000.0,1.0,519.827657,1.0,1.0,1.0,6.059042e+07,17686.0,1286.0,190646.540237,96961.0,8.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [53]:
print(train_df2.shape)
print(train_df2.columns[train_df2.isnull().sum(axis = 0)> 0])

(90275, 92)
Index([], dtype='object')


In [54]:
del train_df, train_df_new, train_df1, one_hot_df
gc.collect()

338

### Begin KNN Analysis 

Now that the data is in a nice format without nulls, we can begin the KNN analysis.  
We will start by scaling the data because KNN is sensitive to the scale of the data. Larger variables will seem more important even if they do not have the correct correlation with the output 

In [55]:
from sklearn.preprocessing import MinMaxScaler

In [56]:
train_df_scale = MinMaxScaler().fit_transform(train_df2.drop(['parcelid', 'transactiondate', 'logerror'],axis = 1))

In [57]:
scale_df_cols = train_df2.drop(['parcelid', 'transactiondate', 'logerror'],axis = 1).columns
train_df3 = pd.DataFrame(train_df_scale, columns = scale_df_cols) 
train_df3.describe()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,...,5,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9,10,11,12,0,1,2,3,4,5,6,7,8,9,10,11,12,13,0
count,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.0,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.0,90275.000000,90275.0,90275.0,90275.0,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,...,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.00000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.0
mean,0.421705,0.113974,0.189492,0.415037,0.068906,0.0,0.172006,0.077892,0.087125,0.660124,0.082056,0.157866,0.293605,0.160413,0.046971,0.065328,0.075500,0.047080,0.451080,0.659818,0.004152,0.0,0.285614,0.0,0.0,0.0,0.163424,0.077011,0.682876,0.242592,0.001989,0.082140,0.002914,0.000778,0.107479,...,0.000089,0.976594,0.000011,0.002382,0.000011,0.000698,0.020305,0.000144,0.424292,0.378787,0.010745,0.171908,0.000022,0.000011,0.000011,0.000842,0.000022,0.000277,0.001074,0.011864,0.000188,0.000011,0.026320,0.006968,0.009737,0.000687,0.671692,0.00093,0.000122,0.003944,0.252728,0.000310,0.025854,0.000510,0.0
std,0.006485,0.050214,0.072277,0.137731,0.051040,0.0,0.023714,0.040668,0.044278,0.003926,0.009568,0.022346,0.013180,0.279236,0.039485,0.050359,0.014607,0.020952,0.179429,0.190499,0.016451,0.0,0.009324,0.0,0.0,0.0,0.277446,0.117548,0.443909,0.138651,0.012054,0.156646,0.012258,0.004514,0.013983,...,0.009413,0.151191,0.003328,0.048744,0.003328,0.026408,0.141041,0.011999,0.494238,0.485088,0.103100,0.377303,0.004707,0.003328,0.003328,0.029003,0.004707,0.016639,0.032762,0.108273,0.013722,0.003328,0.160085,0.083181,0.098195,0.026198,0.469600,0.03049,0.011038,0.062674,0.434579,0.017609,0.158702,0.022568,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.421705,0.100000,0.125000,0.272727,0.052632,0.0,0.172006,0.052113,0.059367,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.319793,0.547388,0.000831,0.0,0.285614,0.0,0.0,0.0,0.002966,0.022785,0.000000,0.242592,0.000695,0.000000,0.002914,0.000000,0.107479,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.421705,0.100000,0.187500,0.415037,0.052632,0.0,0.172006,0

In [58]:
from sklearn.model_selection import train_test_split
Y = train_df2['logerror']
X_train, X_test, y_train, y_test = train_test_split(train_df3,Y, test_size = 0.25)

In [59]:
train_df3.head()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,...,5,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9,10,11,12,0,1,2,3,4,5,6,7,8,9,10,11,12,13,0
0,0.421705,0.100,0.1875,0.272727,0.052632,0.0,0.172006,0.073970,0.084054,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.637696,0.506793,0.001056,0.0,0.285614,0.0,0.0,0.0,0.000075,0.022785,1.0,0.032837,0.001278,0.000000,0.002914,0.000000,0.107479,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.421705,0.175,0.2500,0.415037,0.131579,0.0,0.172006,0.099433,0.112988,0.660124,0.082056,0.157866,0.293605,0.297297,0.046971,0.105263,0.083333,0.063769,0.222673,0.935216,0.000499,0.0,0.285614,0.0,0.0,0.0,0.297009,0.073497,0.0,0.242592,0.003227,0.000000,0.000000,0.000778,0.107479,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.421705,0.150,0.1250,0.272727,0.105263,0.0,0.172006,0.097410,0.110689,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.105263,0.075500,0.047080,0.539723,0.672410,0.001615,0.0,0.285614,0.0,0.0,0.0,0.004907,0.110740,1.0,0.354535,0.001024,0.000000,0.002914,0.000000,0.107479,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.421705,0.100,0.1250,0.272727,0.052632,0.0,0.172006,0.036809,0.041827,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.282049,0.601638,0.010141,0.0,0.285614,0.0,0.0,0.0,0.002641,0.022785,1.0,0.062529,0.000790,0.000000,0.002914,0.000000,0.107479,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.421705,0.125,0.2500,0.415037,0.078947,0.0,0.172006,0.100312,0.113987,0.660124,0.082056,0.157866,0.293605,0.297297,0.046971,0.052632,0.083333,0.081482,0.099104,0.923236,0.000837,0.0,0.285614,0.0,0.0,0.0,0.296872,0.036114,0.0,0.242592,0.003224,0.444444,0.000000,0.000778,0.107479,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
from sklearn.cluster import KMeans

In [61]:
kmeans1 = KMeans(n_clusters = 3).fit(train_df3)

In [62]:
cluster_centers = pd.DataFrame(kmeans1.cluster_centers_, columns = train_df3.columns, index = [1,2,3])

In [63]:
cluster_centers.head()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,...,5,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9,10,11,12,0,1,2,3,4,5,6,7,8,9,10,11,12,13,0
1,0.421705,0.127042,0.192133,0.358515,0.081119,0.0,0.172006,0.082490,0.093730,0.660124,0.082056,0.157866,0.293605,8.332224e-14,0.046971,0.081118,0.075500,0.047079,0.543097,0.619907,0.006821,0.0,0.285614,0.0,0.0,0.0,0.005007,0.075811,1.000000,0.223727,0.001627,0.000010,0.002914,0.000046,0.107479,...,4.351716e-17,0.999974,5.439646e-18,1.994932e-17,2.605592e-05,5.353790e-16,-7.785439e-15,-1.207530e-16,9.980198e-01,2.325917e-13,-1.493597e-15,1.662559e-13,1.087929e-17,5.439646e-18,5.439646e-18,-4.314040e-16,1.087929e-17,9.584347e-17,1.980250e-03,-6.964915e-15,3.363737e-17,5.439646e-18,0.000156,0.000026,-2.376571e-16,0.000391,0.617603,2.605592e-05,2.866151e-04,2.039167e-15,0.324500,0.000417,0.056593,3.278627e-16,0.0
2,0.421705,0.116981,0.196617,0.426462,0.074322,0.0,0.172006,0.083148,0.091084,0.660124,0.082054,0.157866,0.293605,3.980019e-01,0.046979,0.065445,0.075509,0.047079,0.347889,0.708252,0.002338,0.0,0.285614,0.0,0.0,0.0,0.398382,0.070766,0.213454,0.261126,0.002849,0.203690,0.002915,0.001873,0.107479,...,2.199797e-04,0.942530,2.749746e-05,5.444496e-03,5.258381e-18,1.594852e-03,5.040284e-02,3.574669e-04,-1.951217e-13,9.402755e-01,2.667253e-02,3.024720e-04,5.499491e-05,2.749746e-05,2.749746e-05,2.089807e-03,5.499491e-05,6.874364e-04,-1.468010e-16,2.944978e-02,3.574669e-04,2.749746e-05,0.064289,0.017186,2.414277e-02,0.001182,0.612973,2.282289e-03,6.437450e-17,9.789095e-03,0.262271,0.000027,0.004207,1.264883e-03,0.0
3,0.421705,0.074633,0.166278,0.527971,0.026039,0.0,0.172006,0.054218,0.061529,0.660124,0.082063,0.157866,0.293605,4.594716e-04,0.046953,0.026027,0.075480,0.047086,0.465323,0.645030,0.001803,0.0,0.285614,0.0,0.0,0.0,0.004701,0.094604,0.998455,0.245814,0.000870,0.000462,0.002914,0.000019,0.107479,...,5.231275e-18,0.998583,6.539094e-19,1.094726e-03,6.539094e-19,3.219782e-04,8.274631e-15,1.897354e-18,-6.589174e-14,5.567768e-14,-1.392983e-15,9.986477e-01,1.307819e-18,6.539094e-19,6.539094e-19,-1.621966e-16,1.307819e-18,8.050201e-17,1.352309e-03,9.818535e-16,2.575826e-04,6.539094e-19,0.002061,0.000193,6.439565e-05,0.000258,0.942881,1.880007e-16,-1.264451e-17,1.141448e-15,0.052998,0.000708,0.000580,-9.465085e-17,0.0


In [64]:
cluster_centers.T[(cluster_centers.T[1] - cluster_centers.T[2]) > 0.02]

,1,2,3
latitude,0.543097,3.478890e-01,4.653230e-01
regionidcounty,1.000000,2.134537e-01,9.984545e-01
0,0.999974,9.425303e-01,9.985833e-01
1,0.998020,-1.951217e-13,-6.589174e-14
10,0.324500,2.622707e-01,5.299762e-02
12,0.056593,4.207111e-03,5.795608e-04


In [65]:
cluster_centers.T[(cluster_centers.T[1] - cluster_centers.T[3]) > 0.02]

,1,2,3
bathroomcnt,0.127042,1.169811e-01,7.463294e-02
bedroomcnt,0.192133,1.966171e-01,1.662776e-01
calculatedbathnbr,0.081119,7.432201e-02,2.603899e-02
calculatedfinishedsquarefeet,0.082490,8.314811e-02,5.421842e-02
finishedsquarefeet12,0.093730,9.108373e-02,6.152894e-02
fullbathcnt,0.081118,6.544549e-02,2.602729e-02
latitude,0.543097,3.478890e-01,4.653230e-01
yearbuilt,0.682400,6.824653e-01,4.506472e-01
1,0.998020,-1.951217e-13,-6.589174e-14
10,0.324500,2.622707e-01,5.299762e-02


In [66]:
cluster_centers.T[(cluster_centers.T[2] - cluster_centers.T[3]) > 0.02]

,1,2,3
bathroomcnt,1.270421e-01,0.116981,7.463294e-02
bedroomcnt,1.921331e-01,0.196617,1.662776e-01
calculatedbathnbr,8.111915e-02,0.074322,2.603899e-02
calculatedfinishedsquarefeet,8.249028e-02,0.083148,5.421842e-02
finishedsquarefeet12,9.373009e-02,0.091084,6.152894e-02
fips,8.332224e-14,0.398002,4.594716e-04
fullbathcnt,8.111850e-02,0.065445,2.602729e-02
longitude,6.199071e-01,0.708252,6.450299e-01
rawcensustractandblock,5.007101e-03,0.398382,4.700861e-03
roomcnt,1.013286e-05,0.203690,4.615021e-04


A birdie from another script suggested these columns to use instead of the entire dataset. 

This might increase the algorithm performance. 

In [72]:
print(kmeans1.labels_[kmeans1.labels_ == 0].size)
print(kmeans1.labels_[kmeans1.labels_ == 1].size)
print(kmeans1.labels_[kmeans1.labels_ == 2].size)

38379
36367
15529


SyntaxError: invalid syntax (<ipython-input-69-f85cc0acc904>, line 1)

In [38]:
from sklearn import metrics 

In [40]:
homogenity = metrics.homogeneity_score(Y, kmeans1.labels_)
v_meas = metrics.v_measure_score(Y, kmeans1.labels_)
print("Homogenity measure: ", homogenity)
print("V-Score: ", v_meas)

Homogenity measure:  0.00460764837465
V-Score:  0.00804675785851


In [73]:
kmeans2 = KMeans(n_clusters = 2).fit(train_df3)
cluster_centers2 = pd.DataFrame(kmeans2.cluster_centers_, columns = train_df3.columns, index = [1,2])
cluster_centers2.T[(cluster_centers2.T[1] - cluster_centers2.T[2]) > 0.02]

,1,2
latitude,0.520793,3.478554e-01
regionidcounty,1.000000,2.133130e-01
0,0.999981,9.419637e-01
1,0.710842,-1.951772e-13
4,0.287358,9.617763e-04
6,0.711176,6.132285e-01
12,0.040476,4.204336e-03


In [74]:
print(kmeans2.labels_[kmeans2.labels_ == 0].size)
print(kmeans2.labels_[kmeans2.labels_ == 1].size)

53884
36391


In [75]:
print("Cluster 1 Percentage: " , kmeans2.labels_[kmeans2.labels_ == 0].size/ kmeans2.labels_.size)

Percentage of Cluster 1:  0.5968872888396566
